In [44]:
# !pip install langchain
# !pip install openai
# !pip install pydantic
# !pip install transformers
# !pip install pandas
# !pip install numpy
# !pip install gensim
# !pip install faiss-cpu

from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from decimal import Decimal
from transformers import BertTokenizer, BertModel
import csv
import time
import os
import json
import pandas as pd
import numpy as np
import gensim.downloader as api
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation
import faiss
from datetime import datetime

api_key = "OPENAI API KEY"

# Load tokenizer and model from the 'transformers' library
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=api_key)
model = ChatOpenAI(model="gpt-4", openai_api_key=api_key)

prompt_examples = [
  {
        "receipt_text": """
Marley's Shop
123 Long Rd
Kailua, HI 67530
(808) 555-1234
CASHIER: JOHN
REGISTER #: 6
04/12/2023 03:33 PM
Transaction ID: 5769009
PRICE   QTY  TOTAL
APPLES (1 lb)
2.99 2 5.98  
Choco Dream Cookies
7.59 1 7.59   
Member Savings
2.59-
SUBTOTAL
10.98
SALES TAX 8.5%
0.93
TOTAL
11.91
VISA CARD            14.72
CARD#: ************1234
REFERENCE#: 6789
THANK YOU FOR SHOPPING WITH US!""",
      
        "receipt_info": """
{{
  "ReceiptInfo": {{
    "merchant": "Marley's Shop",
    "address": "123 Long Rd",
    "city": "Kailua",
    "state": "HI",
    "phoneNumber": "(808) 555-1234",
    "tax":0.93,
    "total": 10.98,
    "receiptDate": "04/12/2023",
    "receiptTime": "03:33 PM",
    "ITEMS": [
      {{
        "description": "APPLES",
        "quantity": 2,
        "unitPrice": 2.99,
        "totalPrice": 5.98,
        "discountAmount": 0.0
      }},
      {{
        "description": "Choco Dream Cookies",
        "quantity": 1,
        "unitPrice": 7.59,
        "totalPrice": 5,
        "discountAmount": 2.59
      }}
    ]
  }}
}}
"""
    }
#     ,
#     {
#     "receipt_text": """H MART\n...\n458 Keawe st\nHonolulu, <UNKNOWN> 96813\n...\n06/11/23 09:15pm\nWANG DANGMYUN\n4.99 B\nK-T TOPPOKI\nPC\n6.99 B\nOTG SOUP RICECAKE\n7.99 B\nHT VEGE DUMPLING\n14.99 B\nTAX\n1.65\n****\nBALANCE\n36.61\nDiscover Credit - C\n...\nTOTAL AMOUNT: $36.61""",
#     "receipt_info": """
# {{
#   "ReceiptInfo": {{
#     "merchant": "H MART",
#     "address": "458 Keawe st",
#     "city": "Honolulu",
#     "state": "<UNKNOWN>",
#     "phoneNumber": "(808) 219-0924",
#     "tax": 1.65,
#     "total": 36.61,
#     "receiptDate": "06/11/23",
#     "receiptTime": "09:15 PM",
#     "ITEMS": [
#       {{
#         "description": "WANG DANGMYUN",
#         "quantity": 1,
#         "unitPrice": 4.99,
#         "totalPrice": 4.99,
#         "discountAmount": 0.00
#       }},
#       {{
#         "description": "K-T TOPPOKI",
#         "quantity": 1,
#         "unitPrice": 6.99,
#         "totalPrice": 6.99,
#         "discountAmount": 0.00
#       }},
#       {{}
#          "description": "OTG SOUP RICECAKE",
#          "quantity": 1,
#          "unitPrice": 7.99,
#          "totalPrice": 7.99,
#          "discountAmount": 0.00
#       }},
#       {{
#           "description": "HT VEGE DUMPLING",
#           "quantity": 1,
#           "unitPrice": 14.99,
#           "totalPrice": 14.99,
#           "discountAmount": 0.00
#       }}
#     ]
#   }}
# }}
# """
#     },

# {
#     "receipt_text": """
#     SAFEWAY
# O
# <UNKNOWN>
# Store 204 Dir Kaipo Ah Mook Sang
# Main (808) 988-2058
# 2855 East Manoa Road
# HONOLULU HI 96822
# GROCERY
# 6.99 B
# MARINARA SCE
# Regular Price
# 8.49
# Member Savings
# 1.50-
# REFRIG/FROZEN
# LUC CHES COLBY JAC.
# 10.99 B
# Regular Price
# 13.49
# Member Savings
# 2.50-
# MARIE CALLENDER'S
# 4.00 B
# Regular Price
# 6.49
# Member Savings
# 2.49-
# MEAT
# AIDELLS MEATBALLS
# 5.99 B
# Regular Price
# 8.99
# Member Savings
# 3.00-
# TAX
# 1.32
# **** BALANCE
# 29.29
# Credit Purchase 08/01/23 20:15
# CARD # <UNKNOWN>
# <UNKNOWN> 861544421300 AUTH: 0006375D
# PAYMENT AMOUNT
# 29.29
# AL VISA CREDIT
# AID A0000000031010
# TVR 0000000000
# TSI 0000
# Visa
# 29.29
# CHANGE
# 0.00
# TOTAL NUMBER OF ITEMS SOLD =
# 4
# 08/01/23 20:15 204 51 151
# 8851
# YOUR CASHIER TODAY WAS SELF
# Thank <UNKNOWN> you for shopping <UNKNOWN>
# For SAFEWAY FOR U questions
# 877-276-9637 or Safeway.com/foru call""",
    
#     "receipt_info": """
# {{
#   "ReceiptInfo": {{
#     "merchant": "SAFEWAY",
#     "address": "2855 East Manoa Road",
#     "city": "HONOLULU",
#     "state": "HI",
#     "phoneNumber": "(808) 988-2058",
#     "tax": 1.32,
#     "total": 29.29,
#     "receiptDate": "08/01/23",
#     "receiptTime": "20:15",
#     "ITEMS": [
#       {{
#         "description": "MARINARA SCE",
#         "quantity": 1,
#         "unitPrice": 8.49,
#         "totalPrice": 8.49,
#         "discountAmount": 1.50
#       }},
#       {{
#         "description": "LUC CHES COLBY JAC.",
#         "quantity": 1,
#         "unitPrice": 13.49,
#         "totalPrice": 13.49,
#         "discountAmount": 2.50
#       }},
#       {{
#         "description": "MARIE CALLENDER'S",
#         "quantity": 1,
#         "unitPrice": 6.49,
#         "totalPrice": 6.49,
#         "discountAmount": 2.49
#       }},
#       {{
#         "description": "AIDELLS MEATBALLS",
#         "quantity": 1,
#         "unitPrice": 8.99,
#         "totalPrice": 8.99,
#         "discountAmount": 3.00
#       }}
#     ]
#   }}
# }}
# """
# }
]


example_prompt_str =  """
receipt text:
{receipt_text}

{receipt_info}"""

example_prompt = PromptTemplate(
    input_variables=["receipt_text", "receipt_info"],
    template=example_prompt_str
)

class Item(BaseModel):
    description: str = Field(..., description="Item description")
    quantity: int = Field(..., description="Quantity of the item")
    unitPrice: Decimal = Field(..., description="Price per unit")
    totalPrice: Decimal = Field(..., description="Total price for the item")
    discountAmount: Decimal = Field(..., description="Discount amount on the item")

class ReceiptInfo(BaseModel):
    merchant: str = Field(..., description="Merchant name")
    address: str = Field(..., description="Merchant address")
    city: str = Field(..., description="City")
    state: str = Field(..., description="State")
    phoneNumber: str = Field(..., description="Merchant phone number")
    tax: Decimal = Field(..., description="Tax amount")
    total: Decimal = Field(..., description="Total amount")
    receiptDate: str = Field(..., description="Date of the receipt")
    receiptTime: str = Field(..., description="Time of the receipt")
    ITEMS: list[Item] = Field(..., description="List of items purchased")

receiptParser = PydanticOutputParser(pydantic_object=ReceiptInfo)

prompt_prefix = """Extract information from the provided receipt text and format it into a structured JSON.
Try to retrieve as much data as possible from the receipt. For fields where specific information is not available or cannot be extracted, include the key in the JSON with a blank or empty value.
Please note: The total price for each item is calculated as the product of the quantity and the unit price, minus the discount amount. 

{format_instructions}

The receiptDate field should follow the below instruction:
If all data for MONTH, DAY, and YEAR is found:
- Your response should be only the string: 'MONTH/DAY/YEAR'
- The YEAR should have 4 digits like 2023.
If there is a missing data for MONTH, DAY, or YEAR:
- Your response should be only the string: '<UNKNOWN>'.
"""

# print(receiptParser.get_format_instructions())

execute_fewshot_prompt = FewShotPromptTemplate(
    prefix=prompt_prefix,
    input_variables=["receipt_text"],
    partial_variables={"format_instructions": receiptParser.get_format_instructions()},
    examples=prompt_examples,  
    example_prompt=example_prompt, 
    example_separator="\n\n",
    suffix="Receipt text: {receipt_text}\n"
)


def read_text_files(folder_path):
    files_content = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r') as file:
                files_content.append(file.read())
    return files_content 

# Folder containing receipt text files
folder_path = 'receipts'
receipt_texts  = read_text_files(folder_path)
receipt_jsons = []

# Process each receipt text
for idx, receipt_text in enumerate(receipt_texts):
#     if idx == 2 :
#         break
    if (idx + 1) % 3 == 0:
        print("Waiting to avoid rate limit...")
        time.sleep(20)  # Wait for 20 seconds or as needed
#     if idx == 1:
#         break
    data = {"receipt_text": receipt_text}
    
    chain = execute_fewshot_prompt | model
    response = chain.invoke(data)
    receipt_json = response.content
    # response = chain.invoke(data, "temperature": 0.7,"max_tokens": 150)
#     print("Receipt ID: ", idx)
#     print("Receipt Text:\n",receipt_text)
#     print("Receipt Info:\n",receipt_json)
    combined_data = [idx, receipt_text, receipt_json] 
    receipt_jsons.append(combined_data)

# Write data to CSV
with open('receipts_jsons.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    # Write header
    csvwriter.writerow(['Receipt ID','Receipt Text', 'Receipt Info'])
    
    for row in receipt_jsons:
        csvwriter.writerow(row)



csv_file_path = 'receipts_jsons.csv'
# csv_file_path = 'receipts_json_original.csv'

jsons_df = pd.read_csv(csv_file_path)
  
receipt_data = []
item_data = []

for index, row in jsons_df.iterrows():
    receipt_id = row['Receipt ID']
    receipt_text = row['Receipt Text']
    receipt_json = row['Receipt Info']
    
#     print("Receipt No. ", receipt_id)
#     print("Receipt Text:\n",receipt_text)
#     print("Receipt Info:\n",receipt_json)

    
    json_data = json.loads(receipt_json)
    receipt_info = json_data["ReceiptInfo"]
    basic_info = [receipt_info.get(key, '') for key in ["merchant", "address", "city", "state", "phoneNumber", "tax", "total", "receiptDate", "receiptTime"]]
    
    combined_data = [receipt_id, receipt_text, receipt_json,receipt_category] + basic_info 
    receipt_data.append(combined_data)
    
    items = receipt_info["ITEMS"]
    for i, item in enumerate(items, 1):
        item_info = [item.get(key, '') for key in ["description", "quantity", "unitPrice", "totalPrice","discountAmount"]]
        item_data.append( [receipt_id,receipt_category] + item_info)
#     break

# Write data to CSV
with open('receipts_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    # Write header
    csvwriter.writerow(['Receipt ID','Receipt Text', 'Receipt Info',"Receipt Category","Merchant", "Address", "City", "State", "PhoneNumber", "Tax", "Total", "Receipt Date", "Receipt Time"])
    
    for row in receipt_data:
        csvwriter.writerow(row)
        
# Write data to CSV
with open('items_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    # Write header
    csvwriter.writerow(['Receipt ID','Item Category',"Description", "Quantity", "Unit Price", "Total Price","Discount Amount"])
    
    for row in item_data:
        csvwriter.writerow(row)
        

df = pd.read_excel('final_categories.xlsx')  # Replace with your file path
# df = pd.read_excel('Category_Descriptions.xlsx')  # Replace with your file path

def preprocess(text):
#     print("Before",text)
    if isinstance(text,float):
        return text
    text = text.lower()
    text = strip_punctuation(text)
    text = remove_stopwords(text)
    return text

df['processed_descriptions'] = df['Category Descriptions'].apply(preprocess)
# print(df['processed_descriptions'])


# # Load tokenizer and model from the 'transformers' library
# bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_model = BertModel.from_pretrained('bert-base-uncased')


def get_bert_embedding(text):
    
    if not text:
        print("Invalid input: Expected a non-empty string.")
        return np.zeros((1, 768))  # Return a zero vector for invalid inputs

    # Tokenize input text and convert to PyTorch tensors
    inputs = bert_tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    # Generate embeddings using BERT model
    outputs = bert_model(**inputs)
    # Use the mean of the last hidden states as the sentence embedding
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings[0]

# df['embeddings'] = df['processed_descriptions'].apply(get_bert_embedding)

faiss_index = faiss.read_index("faiss_index.index")

# dim =768  # Dimension of vectors
# faiss.normalize_L2(np.array(df['embeddings'].tolist()))

# # Use IndexFlatIP for cosine similarity
# faiss_index = faiss.IndexFlatIP(dim) 
# faiss_index.add(np.array(df['embeddings'].tolist()).astype('float32'))

# faiss.write_index(faiss_index, "faiss_index.index")


def categorize_receipt(receipt_description):
    processed = preprocess(receipt_description)
    embedding = get_bert_embedding(processed).reshape(1, -1).astype('float32')
    distances, indices = faiss_index.search(embedding, 1)
    
    nearest_neighbor_index = indices[0][0]
    
    # Retrieve the actual description from the DataFrame
    nearest_neighbor_description = df.iloc[nearest_neighbor_index]['processed_descriptions']
    
    # Retrieve the category of the nearest neighbor
    nearest_neighbor_category = df.iloc[nearest_neighbor_index]['Receipt Category']
    
    return nearest_neighbor_category, nearest_neighbor_description, nearest_neighbor_index


# Load CSV files
receipts_df = pd.read_csv('receipts_data.csv')
items_df = pd.read_csv('items_data.csv')

# Create a dictionary to store the concatenated strings

# Iterate over each row in the receipts DataFrame
for index, row in receipts_df.iterrows():
    receipt_id = row['Receipt ID']
    merchant_name = row['Merchant']
    receipt_text = row['Receipt Text']
    receipt_info = row['Receipt Info']


    # Filter items belonging to this receipt
    filtered_items = items_df[items_df['Receipt ID'] == receipt_id]

    # Concatenate merchant name with item descriptions
    items_string = ' '.join(filtered_items['Description'].tolist())
    receipt_description = f"{merchant_name} {items_string}".replace('<UNKNOWN>', '')
    category, neighbor_description, neighbor_index = categorize_receipt(receipt_description)        
    receipts_df.at[index, 'Predicted Category'] = category
#     jsons_df.at[receipt_id, 'Receipt Category'] = category

    
    print("Receipt Number :", index)
    print("Receipt Text :\n", receipt_text)
    print("Receipt Info :\n", receipt_info)
    print("Receipt Description:", receipt_description)

    print("Category:", category)
    print("Nearest Neighbor Description:", neighbor_description)
    print("Nearest Neighbor Index:", neighbor_index)
    print("\n" + "-"*50 + "\n")  # Separator between rows


    
for index, row in items_df.iterrows():
    item_description = row['Description'].replace('<UNKNOWN>', '')
    category, neighbor_description, neighbor_index = categorize_receipt(item_description)        
    items_df.at[index, 'Predicted Category'] = category
    print("Item from Receipt Number :", row['Receipt ID'])
    print("Item Description:", item_description)
    print("Category:", category)
    print("Nearest Neighbor Description:", neighbor_description)
    print("Nearest Neighbor Index:", neighbor_index)
    print("\n" + "-"*50 + "\n")  # Separator between rows

        
# Optionally, save the updated DataFrame to a new CSV file
# receipts_df.to_csv('updated_receipts_data.csv', index=False)
# items_df.to_csv('updated_items_data.csv', index=False)

# receipts_df = pd.read_csv('receipts_data.csv')
# items_df = pd.read_csv('items_data.csv')
synthetic_items_df = items_df[['Description', 'Item Category', 'Unit Price', 'Discount Amount']].copy()
synthetic_items_df.columns = ['Item Name', 'Item Category', 'Unit Price ($)', 'Discount ($)']



synthetic_items_df.to_csv('items_dataset.csv', index=False)

def convert_to_float_and_abs_or_zero(value):
    try:
        return abs(float(value))
    except ValueError:
        return 0.0

items_df['Discount Amount'] = items_df['Discount Amount'].apply(convert_to_float_and_abs_or_zero)


aggregated_items = items_df.groupby('Receipt ID').agg(
    Number_of_Items=('Receipt ID', 'count'),
    Discount_Amount=('Discount Amount', 'sum')
).reset_index()

# Step 2: Merge with receipts_df
merged_df = pd.merge(
    receipts_df,
    aggregated_items,
    on='Receipt ID',
    how='left'
)

# Step 3: Select and rename columns
synthetic_receipts_df = merged_df[['Merchant', 'Number_of_Items', 'Total', 'Discount_Amount', 'Tax', 'Predicted Category']]
synthetic_receipts_df.columns = ['Merchant', 'Number of Items', 'Total Price', 'Discount Amount', 'Tax', 'Category']

# Step 4: Write to CSV
synthetic_receipts_df.to_csv('receipts_dataset.csv', index=False)

process_table_df = pd.read_csv('receipts_jsons.csv')
receipts_df = pd.read_csv('receipts_dataset.csv')
items_df = pd.read_csv('items_dataset.csv')

# To change the dataset for the dashboard process table
records = []
for i in range(len(process_table_df)):

    receipt_text = process_table_df.iloc[i]['Receipt Text']

    rec = json.loads(process_table_df.iloc[i]['Receipt Info'])
    receipt_json = json.dumps(rec, indent=4)

    merhcant_name = rec['ReceiptInfo']['merchant']
    merhcant_category = receipts_df[receipts_df['Merchant'] == merhcant_name].iloc[0]['Category']

    items_arr = []
    for item in rec['ReceiptInfo']['ITEMS']:
        item_name = item['description']
        item_category = items_df[items_df['Item Name'] == item_name].iloc[0]['Item Category']

        items_arr.append({'Name': item_name, 'Category': item_category})
    
    receipt_items = json.dumps(items_arr, indent=4)

    record = [receipt_text, receipt_json, merhcant_name, merhcant_category, receipt_items]
    records.append(record)

columns = ['Receipt Text', 'Receipt JSON', 'Merchant Name', 'Merchant Category', 'Items']

df_process_table = pd.DataFrame(records, columns=columns)
receipts_dataset_path = 'datasets/process_table.csv'
df_process_table.to_csv(receipts_dataset_path, index=False)

# To reformat time to HOUR:MINUTE:SECOND
def time_reformater(receipt_time):
    receipt_time = receipt_time.lower()

    if 'am' in receipt_time:
        receipt_time = receipt_time.replace('am', '')
    if 'a' in receipt_time:
        receipt_time = receipt_time.replace('a', '')
    if 'pm' in receipt_time:
        receipt_time = receipt_time.replace('pm', '')
        hour = receipt_time.split(':')[0]
        mins = receipt_time.split(':')[1]
        if hour != 12:
            receipt_time = f"{int(hour)+12}:{mins}"
    if 'p' in receipt_time:
        receipt_time = receipt_time.replace('p', '')
        hour = receipt_time.split(':')[0]
        mins = receipt_time.split(':')[1]
        if hour != 12:
            receipt_time = f"{int(hour)+12}:{mins}"

    if receipt_time.count(':') == 1:
        receipt_time += ':00'
    
    receipt_time = receipt_time.strip()
    
    return receipt_time

# To create the vendors dataset
records = []
for i in range(len(process_table_df)):
    rec = json.loads(process_table_df.iloc[i]['Receipt Info'])
    merchant_name = rec['ReceiptInfo']['merchant']
    merhcant_category = receipts_df[receipts_df['Merchant'] == merhcant_name].iloc[0]['Category']

    number_of_items = 0
    discount = 0
    for item in rec['ReceiptInfo']['ITEMS']:
        # for the number of items
        try:
            number_of_items += item['quantity']
        except:
            number_of_items += 1

        # for the discount
        try:
            discount += float(item['discountAmount'])
        except:
            pass

    total_price = rec['ReceiptInfo']['total']
    tax = rec['ReceiptInfo']['tax']

    receipt_date = rec['ReceiptInfo']['receiptDate']
    receipt_time = time_reformater(rec['ReceiptInfo']['receiptTime'])
    receipt_hour = receipt_time.split(':')[0]

    record = [merchant_name, merchant_category, number_of_items, total_price, discount, tax, receipt_date, receipt_time, receipt_hour]
    records.append(record)

columns = ['Vendor Name', 'Category', 'Number of Items', 'Total Price ($)', 'Discount ($)', 'Tax ($)', 'Date', 'Time', 'Hour']

df_vendors = pd.DataFrame(records, columns=columns)
vendors_dataset_path = 'datasets/vendors_dataset.csv'
df_vendors.to_csv(vendors_dataset_path, index=False)

# To create the items dataset
records = []

for i in range(len(process_table_df)):
    rec = json.loads(process_table_df.iloc[i]['Receipt Info'])

    item_date = rec['ReceiptInfo']['receiptDate']
    item_time = time_reformater(rec['ReceiptInfo']['receiptTime'])
    item_hour = item_time.split(':')[0]

    for item in rec['ReceiptInfo']['ITEMS']:
        item_name = item['description']
        item_unit_price = item['unitPrice']
        item_discount = item['discountAmount']

        item_category = items_df[items_df['Item Name'] == item_name].iloc[0]['Item Category']

        record = [item_name, item_category, item_unit_price, item_discount, item_date, item_time, item_hour]
        records.append(record)

columns = ['Item Name', 'Item Category', 'Unit Price ($)', 'Discount ($)', 'Date', 'Time', 'Hour']

df_items = pd.DataFrame(records, columns=columns)
items_dataset_path = 'datasets/items_dataset.csv'
df_items.to_csv(items_dataset_path, index=False)


Receipt Number : 0
Receipt Text :
 THANK YOU FOR SHOPPING AT
BEN FRANKLIN CRAFTS #05575 (2)
(808) 261-4621
08/31/23 7:22PM AZRIROS
106
SALE
30149676
2
EA
$34.09 EA
LAUHALA MAT <UNKNOWN>
$68.18
SUB-TOTAL:$
68.18 TAX: $
2.89
DISCOUNT:
-6.82 TOTAL: $
64.25
BC AMT:
$
64.25

Receipt Info :
 {
  "ReceiptInfo": {
    "merchant": "BEN FRANKLIN CRAFTS #05575",
    "address": "",
    "city": "",
    "state": "",
    "phoneNumber": "(808) 261-4621",
    "tax": 2.89,
    "total": 64.25,
    "receiptDate": "<UNKNOWN>",
    "receiptTime": "7:22PM",
    "ITEMS": [
      {
        "description": "LAUHALA MAT",
        "quantity": 2,
        "unitPrice": "$34.09",
        "totalPrice": "$68.18",
        "discountAmount": "-6.82"
      }
    ]
  }
}
Receipt Description: BEN FRANKLIN CRAFTS #05575 LAUHALA MAT
Category: Entertainment and Leisure
Nearest Neighbor Description: entertainment audiobooks movie posters movie
Nearest Neighbor Index: 689

--------------------------------------------------

Receip